In [1]:
import tensorflow as tf
import librosa
import torch
from models.TMC import ETMC, ce_loss, TMC
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
from models.image import ImageEncoder

c:\Users\debonair\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\debonair\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
c:\Users\debonair\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Users\debonair\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
FEATURE_DESCRIPTION = {
    'video_path': tf.io.FixedLenFeature([], tf.string), 
    'image/encoded': tf.io.FixedLenFeature([], tf.string),
    'clip/label/index': tf.io.FixedLenFeature([], tf.int64),
    'clip/label/text': tf.io.FixedLenFeature([], tf.string), 
    'WAVEFORM/feature/floats': tf.io.FixedLenFeature([], tf.string)
}

@tf.function
def _parse_function(example_proto):

    #Parse the input `tf.train.Example` proto using the dictionary above.
    example = tf.io.parse_single_example(example_proto, FEATURE_DESCRIPTION)
    
    video_path = example['video_path']
    video = tf.io.decode_raw(example['image/encoded'], tf.int8)
    video_reshaped = video.reshape(10, 256, 256, 3)
    
    spectrogram = tf.io.decode_raw(example['WAVEFORM/feature/floats'], tf.float32)
    spectrogram = spectrogram
    
    label = example["clip/label/text"]
    label_map = example["clip/label/index"]
    
    return video_path, video_reshaped, spectrogram, label, label_map

class FakeAVCelebDataset:

    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.samples = self.load_features_from_tfrec()

    def load_features_from_tfrec(self):
        '''Loads raw features from a tfrecord file and returns them as raw inputs'''
        ds = tf.io.matching_files(self.data_dir)
        files = tf.random.shuffle(ds)
        shards = tf.data.Dataset.from_tensor_slices(files)
        dataset = shards.interleave(tf.data.TFRecordDataset)
        dataset = dataset.shuffle(buffer_size=1000)
        dataset = dataset.map(_parse_function, num_parallel_calls = tf.data.AUTOTUNE)
        return dataset
    
    def __len__(self):
        self.samples = self.load_features_from_tfrec(self.data_dir)
        cnt = self.samples.reduce(np.int64(0), lambda x, _: x + 1)
        cnt = cnt.numpy()
        return cnt



In [3]:
ds = FakeAVCelebDataset(data_dir='./datasets/fakeav*')
ds = ds.load_features_from_tfrec()
ds.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(10, 256, 256, 3), dtype=tf.int8, name=None),
 TensorSpec(shape=(None,), dtype=tf.float32, name=None),
 TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [4]:
for element in ds:
    video_path, video_reshaped, spectrogram, label, label_map = element
    video_frame = video_reshaped[0].numpy()
    print(video_frame.shape, label_map.numpy(), spectrogram.shape)
    break

(256, 256, 3) 0 (132864,)


In [5]:
spec_shape_list = []
for element in ds:
    video_path, video_reshaped, spectrogram, label, label_map = element
    spec_shape = spectrogram.numpy().reshape(-1, 256).shape
    spec_shape_list.append(spec_shape[0])

In [6]:
max(spec_shape_list), min(spec_shape_list)

(1353, 83)

In [7]:
import tensorflow as tf

# Create a tensor
input_tensor = tf.random.uniform([293, 256], 0, 1, tf.float32)

MAX_SPEC = max(spec_shape_list)

# # Calculate the necessary paddings
pad_tensor_height = MAX_SPEC - input_tensor.shape[0]

print(pad_tensor_height)

# pad_width = max(256 - tf.shape(input_tensor)[1], 0)


# print(pad_height, pad_width)

# # Pad the tensor
padded_tensor = tf.pad(input_tensor, paddings=[[0, pad_tensor_height], [0, 0]])

# Print the padded tensor shape
print(padded_tensor.shape, input_tensor.shape)


1060
(1353, 256) (293, 256)


In [8]:
vid = video_reshaped.numpy()
spec = padded_tensor.numpy()

vid_pt = torch.Tensor(vid)
spec_pt = torch.Tensor(spec)

vid_pt.shape, spec_pt.shape

(torch.Size([10, 256, 256, 3]), torch.Size([1353, 256]))

In [9]:
vid_pt_permuted = vid_pt.permute(0, 3, 2, 1)
vid_pt_permuted.shape

torch.Size([10, 3, 256, 256])

In [10]:
import torchvision
import torch.nn as nn

class ImageEncoder(nn.Module):
    def __init__(self):
        super(ImageEncoder, self).__init__()
        # self.args = args

        model = torchvision.models.resnet18(pretrained=False)
        modules = list(model.children())[:-1]
        self.model = nn.Sequential(*modules)
        pool_func = nn.AdaptiveAvgPool2d

        # if args.num_image_embeds in [1, 2, 3, 5, 7]:
        self.pool = pool_func((2, 1))
        # elif args.num_image_embeds == 4:
        #     self.pool = pool_func((2, 2))
        # elif args.num_image_embeds == 6:
        #     self.pool = pool_func((3, 2))
        # elif args.num_image_embeds == 8:
        #     self.pool = pool_func((4, 2))
        # elif args.num_image_embeds == 9:
        #     self.pool = pool_func((3, 3))

    def forward(self, x):
        # Bx3x224x224 -> Bx2048x7x7 -> Bx2048xN -> BxNx2048
        out = self.model(x)
        out = self.pool(out)
        out = torch.flatten(out, start_dim=2)
        out = out.transpose(1, 2).contiguous()
        return out  # BxNx2048

In [11]:
y = torch.rand((1, 1, 256, 256))
conv = nn.Sequential(nn.Conv2d(in_channels=1, out_channels=3, kernel_size=(2, 2)))

out = conv(y)
out.shape

torch.Size([1, 3, 255, 255])

In [41]:
enc = ImageEncoder()

out = enc(vid_pt_permuted)
out.shape

c:\Users\debonair\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\debonair\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


torch.Size([10, 2, 512])

In [42]:
vid_out = out.reshape(shape = (10, 1024))
vid_out.shape

torch.Size([10, 1024])

In [13]:
import librosa
x, sr = librosa.load('my_result.wav')
x.shape

(95697,)

In [62]:
from models.rawnet import SincConv, Residual_block
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
import numpy as np
from torch.utils import data
from collections import OrderedDict
from torch.nn.parameter import Parameter

audio_args = {
  'nb_samp': 64600,
  'first_conv': 1024,
  'in_channels': 1,
  'filts': [20, [20, 20], [20, 128], [128, 128]],
  'blocks': [2, 4],
  'nb_fc_node': 1024,
  'gru_node': 1024,
  'nb_gru_layer': 3,
  'nb_classes': 2
}
class RawNet(nn.Module):
    def __init__(self, d_args, device):
        super(RawNet, self).__init__()

        
        self.device=device

        self.Sinc_conv=SincConv(device=self.device,
			out_channels = d_args['filts'][0],
			kernel_size = d_args['first_conv'],
                        in_channels = d_args['in_channels']
        )
        
        self.first_bn = nn.BatchNorm1d(num_features = d_args['filts'][0])
        self.selu = nn.SELU(inplace=True)
        self.block0 = nn.Sequential(Residual_block(nb_filts = d_args['filts'][1], first = True))
        self.block1 = nn.Sequential(Residual_block(nb_filts = d_args['filts'][1]))
        self.block2 = nn.Sequential(Residual_block(nb_filts = d_args['filts'][2]))
        d_args['filts'][2][0] = d_args['filts'][2][1]
        self.block3 = nn.Sequential(Residual_block(nb_filts = d_args['filts'][2]))
        self.block4 = nn.Sequential(Residual_block(nb_filts = d_args['filts'][2]))
        self.block5 = nn.Sequential(Residual_block(nb_filts = d_args['filts'][2]))
        self.avgpool = nn.AdaptiveAvgPool1d(1)

        self.fc_attention0 = self._make_attention_fc(in_features = d_args['filts'][1][-1],
            l_out_features = d_args['filts'][1][-1])
        self.fc_attention1 = self._make_attention_fc(in_features = d_args['filts'][1][-1],
            l_out_features = d_args['filts'][1][-1])
        self.fc_attention2 = self._make_attention_fc(in_features = d_args['filts'][2][-1],
            l_out_features = d_args['filts'][2][-1])
        self.fc_attention3 = self._make_attention_fc(in_features = d_args['filts'][2][-1],
            l_out_features = d_args['filts'][2][-1])
        self.fc_attention4 = self._make_attention_fc(in_features = d_args['filts'][2][-1],
            l_out_features = d_args['filts'][2][-1])
        self.fc_attention5 = self._make_attention_fc(in_features = d_args['filts'][2][-1],
            l_out_features = d_args['filts'][2][-1])

        self.bn_before_gru = nn.BatchNorm1d(num_features = d_args['filts'][2][-1])
        self.gru = nn.GRU(input_size = d_args['filts'][2][-1],
			hidden_size = d_args['gru_node'],
			num_layers = d_args['nb_gru_layer'],
			batch_first = True)

        
        self.fc1_gru = nn.Linear(in_features = d_args['gru_node'],
			out_features = d_args['nb_fc_node'])
       
        self.fc2_gru = nn.Linear(in_features = d_args['nb_fc_node'],
			out_features = d_args['nb_classes'],bias=True)
			
       
        self.sig = nn.Sigmoid()
        self.logsoftmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, y = None):
        
        
        nb_samp = x.shape[0]
        len_seq = x.shape[1]
        print(nb_samp, len_seq)
        x=x.view(nb_samp,1,len_seq)
        
        x = self.Sinc_conv(x)    
        x = F.max_pool1d(torch.abs(x), 3)
        x = self.first_bn(x)
        x =  self.selu(x)
        
        x0 = self.block0(x)
        y0 = self.avgpool(x0).view(x0.size(0), -1) # torch.Size([batch, filter])
        y0 = self.fc_attention0(y0)
        y0 = self.sig(y0).view(y0.size(0), y0.size(1), -1)  # torch.Size([batch, filter, 1])
        x = x0 * y0 + y0  # (batch, filter, time) x (batch, filter, 1)
        

        x1 = self.block1(x)
        y1 = self.avgpool(x1).view(x1.size(0), -1) # torch.Size([batch, filter])
        y1 = self.fc_attention1(y1)
        y1 = self.sig(y1).view(y1.size(0), y1.size(1), -1)  # torch.Size([batch, filter, 1])
        x = x1 * y1 + y1 # (batch, filter, time) x (batch, filter, 1)

        x2 = self.block2(x)
        y2 = self.avgpool(x2).view(x2.size(0), -1) # torch.Size([batch, filter])
        y2 = self.fc_attention2(y2)
        y2 = self.sig(y2).view(y2.size(0), y2.size(1), -1)  # torch.Size([batch, filter, 1])
        x = x2 * y2 + y2 # (batch, filter, time) x (batch, filter, 1)

        x3 = self.block3(x)
        y3 = self.avgpool(x3).view(x3.size(0), -1) # torch.Size([batch, filter])
        y3 = self.fc_attention3(y3)
        y3 = self.sig(y3).view(y3.size(0), y3.size(1), -1)  # torch.Size([batch, filter, 1])
        x = x3 * y3 + y3 # (batch, filter, time) x (batch, filter, 1)

        x4 = self.block4(x)
        y4 = self.avgpool(x4).view(x4.size(0), -1) # torch.Size([batch, filter])
        y4 = self.fc_attention4(y4)
        y4 = self.sig(y4).view(y4.size(0), y4.size(1), -1)  # torch.Size([batch, filter, 1])
        x = x4 * y4 + y4 # (batch, filter, time) x (batch, filter, 1)

        x5 = self.block5(x)
        y5 = self.avgpool(x5).view(x5.size(0), -1) # torch.Size([batch, filter])
        y5 = self.fc_attention5(y5)
        y5 = self.sig(y5).view(y5.size(0), y5.size(1), -1)  # torch.Size([batch, filter, 1])
        x = x5 * y5 + y5 # (batch, filter, time) x (batch, filter, 1)

        x = self.bn_before_gru(x)
        x = self.selu(x)
        x = x.permute(0, 2, 1)     #(batch, filt, time) >> (batch, time, filt)
        self.gru.flatten_parameters()
        x, _ = self.gru(x)
        x = x[:,-1,:]
        # output = self.fc1_gru(x)
        # x = self.fc2_gru(x)
        # output=self.logsoftmax(x)
      
        return x
        
        

    def _make_attention_fc(self, in_features, l_out_features):

        l_fc = []
        
        l_fc.append(nn.Linear(in_features = in_features,
			        out_features = l_out_features))

        

        return nn.Sequential(*l_fc)


    def _make_layer(self, nb_blocks, nb_filts, first = False):
        layers = []
        #def __init__(self, nb_filts, first = False):
        for i in range(nb_blocks):
            first = first if i == 0 else False
            layers.append(Residual_block(nb_filts = nb_filts,
				first = first))
            if i == 0: nb_filts[0] = nb_filts[1]
            
        return nn.Sequential(*layers)


rawnet_model = RawNet(d_args = audio_args, device = torch.device('cpu'))

In [63]:
out = rawnet_model(torch.unsqueeze(torch.Tensor(x), dim = 0))
out.shape

1 95697


torch.Size([1, 1024])

In [110]:
spec_pt.shape, torch.unsqueeze(spec_pt, dim = 0).shape

(torch.Size([1353, 256]), torch.Size([1, 1353, 256]))

In [43]:
import argparse
from models.TMC import ETMC, ce_loss
def get_args(parser):
    parser.add_argument("--batch_sz", type=int, default=32)
    parser.add_argument("--train_data_path", type=str, default="datasets/train")
    parser.add_argument("--val_data_path", type=str, default="datasets/val")
    parser.add_argument("--LOAD_SIZE", type=int, default=256)
    parser.add_argument("--FINE_SIZE", type=int, default=224)
    parser.add_argument("--dropout", type=float, default=0.1)
    parser.add_argument("--gradient_accumulation_steps", type=int, default=3)
    parser.add_argument("--hidden", nargs="*", type=int, default=[])
    parser.add_argument("--hidden_sz", type=int, default=768)
    parser.add_argument("--img_embed_pool_type", type=str, default="avg", choices=["max", "avg"])
    parser.add_argument("--img_hidden_sz", type=int, default=512)
    parser.add_argument("--include_bn", type=int, default=True)
    parser.add_argument("--lr", type=float, default=1e-4)
    parser.add_argument("--lr_factor", type=float, default=0.3)
    parser.add_argument("--lr_patience", type=int, default=10)
    parser.add_argument("--max_epochs", type=int, default=500)
    parser.add_argument("--n_workers", type=int, default=12)
    parser.add_argument("--name", type=str, default="ReleasedVersion")
    parser.add_argument("--num_image_embeds", type=int, default=1)
    parser.add_argument("--patience", type=int, default=20)
    parser.add_argument("--savedir", type=str, default="./savepath/ETMC/nyud2/")
    parser.add_argument("--seed", type=int, default=1)
    parser.add_argument("--n_classes", type=int, default=10)
    parser.add_argument("--annealing_epoch", type=int, default=10)

parser = argparse.ArgumentParser(description="Train Models")
get_args(parser)
args, remaining_args = parser.parse_known_args()
model = ETMC(args)

c:\Users\debonair\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\debonair\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [102]:
out = rawnet_model(torch.unsqueeze(spec_pt, dim = 0))
out.shape

RuntimeError: shape '[1, 1, 1353]' is invalid for input of size 346368